In [ ]:
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.constants import SPATIAL, NODATAINT, MAX, SQUARE, EXACT
from geopyspark.geotrellis.rdd import TiledRasterRDD

import json
import shapely
from shapely.geometry import shape, asShape, MultiPoint, MultiLineString

In [ ]:
geopysc = GeoPyContext(appName="nlcd", master="local[*]")

# View NLCD from GeoTrellis Catalog

In [ ]:
from geopyspark.geotrellis.tms import *
from geopyspark.geotrellis.color import ColorMap
from geonotebook.wrappers.raster import TMSRasterData

nlcd_layer_name = "nlcd-tms-epsg3857"

nlcd = s3_catalog_tms_server(
    geopysc, 
    "azavea-datahub", "catalog", 
    nlcd_layer_name, 
    ColorMap.nlcd_colormap(geopysc))

In [ ]:
M.add_layer(TMSRasterData(nlcd), name="nlcd")

# Read Chattanooga NLCD Tiles

In [ ]:
M.set_center(-85.2934168635424, 35.02445474101138, 9)

In [ ]:
!curl -o /tmp/mask.json https://s3.amazonaws.com/chattademo/chatta_mask.json

from geonotebook.wrappers import VectorData
vd = VectorData("/tmp/mask.json")
name = "Outline"
M.add_layer(vd, name=name)

In [ ]:
from functools import partial
import fiona
import json
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

txt = open('/tmp/mask.json').read()
js = json.loads(txt)
geom = shape(js)
center = geom.centroid
chatta_poly = transform(project, geom)
chatta_poly

In [ ]:
from geopyspark.geotrellis import catalog

MAX_ZOOM = 12
query_rdd = catalog.query(
    geopysc, SPATIAL, 
    "s3://azavea-datahub/catalog", nlcd_layer_name, 
    MAX_ZOOM, intersects=chatta_poly)

chatta_rdd = query_rdd.convert_data_type("int8").cache()

## Work with Chattanooga RDD

In [ ]:
chatta_rdd.get_min_max()

In [ ]:
chatta_rdd.layer_metadata.extent

In [ ]:
chatta_py_rdd = chatta_rdd.to_numpy_rdd()